In [ ]:
from imports import *

In [2]:
tf.__version__

'1.1.0'

In [3]:
import cleaning_data as clean_data

(568454, 10)
   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality can

In [4]:
import preprocessing as pre

# Clean the summaries and texts
clean_summaries = []
for summary in clean_data.reviews.Summary:
    clean_summaries.append(pre.clean_text(summary, remove_stopwords=False))
print("Summaries are complete.")

clean_texts = []
for text in clean_data.reviews.Text:
    clean_texts.append(pre.clean_text(text))
print("Texts are complete.")


Summaries are complete.
Texts are complete.


In [5]:
for i in range(5):
    print("Clean Review #",i+1)
    print(clean_summaries[i])
    print(clean_texts[i])
    print()

Clean Review # 1
good quality dog food
bought several vitality canned dog food products found good quality product looks like stew processed meat smells better labrador finicky appreciates product better

Clean Review # 2
not as advertised
product arrived labeled jumbo salted peanuts peanuts actually small sized unsalted sure error vendor intended represent product jumbo

Clean Review # 3
 delight  says it all
confection around centuries light pillowy citrus gelatin nuts case filberts cut tiny squares liberally coated powdered sugar tiny mouthful heaven chewy flavorful highly recommend yummy treat familiar story c lewis lion witch wardrobe treat seduces edmund selling brother sisters witch

Clean Review # 4
cough medicine
looking secret ingredient robitussin believe found got addition root beer extract ordered good made cherry soda flavor medicinal

Clean Review # 5
great taffy
great taffy great price wide assortment yummy taffy delivery quick taffy lover deal



In [6]:
#make sure you have a folder named cleaned_data in the current directory
import pickle as cPickle
rff = open('./cleaned_data/clean_summaries.pkl', 'wb')
cPickle.dump(clean_summaries, rff)
rff.close()

rff = open('./cleaned_data/clean_texts.pkl', 'wb')
cPickle.dump(clean_texts, rff)
rff.close()


In [7]:
rf_pkl = open('./cleaned_data/clean_summaries.pkl', 'rb')
clean_summaries=cPickle.load(rf_pkl)
rf_pkl.close()

rf_pkl = open('./cleaned_data/clean_texts.pkl', 'rb')
clean_texts=cPickle.load(rf_pkl)
rf_pkl.close()


In [8]:

embeddings_index = {}
#with open('embeddings/numberbatch-17.06.txt', encoding='utf-8') as f:
with open('embeddings/numberbatch-en-17.02.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))


Word embeddings: 484557


In [9]:
from vocabulary import *

word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))


Size of Vocabulary: 132884


In [10]:
missing_words = 0
threshold = 15

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))


Number of words missing from CN: 3941
Percent of words that are missing from vocabulary: 2.97%


In [11]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 132884
Number of words we will use: 66450
Percent of words we will use: 50.01%


In [12]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

66450


In [13]:

def convert_to_ints(text, word_count, unk_count,eos=False):
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count


In [14]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 25679946
Total number of UNKs in headlines: 154085
Percent of words that are UNK: 0.6%


In [15]:
lns=[]
for sentence in int_summaries:
    lns.append(len(sentence))
    
lengths_summaries=pd.DataFrame(lns, columns=['counts'])

lns=[]
for sentence in int_texts:
    lns.append(len(sentence))
    
lengths_texts=pd.DataFrame(lns, columns=['counts'])


In [16]:
print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

print
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))
print
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))



Summaries:
              counts
count  568412.000000
mean        4.181620
std         2.657872
min         0.000000
25%         2.000000
50%         4.000000
75%         5.000000
max        48.000000

Texts:
              counts
count  568412.000000
mean       41.996782
std        42.520854
min         1.000000
25%        18.000000
50%        29.000000
75%        50.000000
max      2085.000000
84.0
115.0
207.0
8.0
9.0
13.0


In [17]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

# Sort the summaries and texts by the length of the texts, shortest to longest
# Limit the length of summaries and texts based on the min and max ranges.
# Remove reviews that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 84
max_summary_length = 13
min_length = 2
unk_text_limit = 1
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

431562
431562


In [18]:
#make sure you have a folder named cleaned_data in the current directory
import pickle as cPickle
rff = open('./cleaned_data/sorted_summaries_ints.pkl', 'wb')
cPickle.dump(sorted_summaries, rff)
rff.close()

rff = open('./cleaned_data/sorted_texts_ints.pkl', 'wb')
cPickle.dump(sorted_texts, rff)
rff.close()


In [19]:
rf_pkl = open('./cleaned_data/sorted_summaries_ints.pkl', 'rb')
sorted_summaries=cPickle.load(rf_pkl)
rf_pkl.close()

rf_pkl = open('./cleaned_data/sorted_texts_ints.pkl', 'rb')
sorted_texts=cPickle.load(rf_pkl)
rf_pkl.close()


In [20]:
#print (vocab_to_int)
#print (int_to_vocab)

#make sure you have a folder named cleaned_data in the current directory
import pickle as cPickle
rff = open('./cleaned_data/vocab_to_int.pkl', 'wb')
cPickle.dump(vocab_to_int, rff)
rff.close()

rff = open('./cleaned_data/int_to_vocab.pkl', 'wb')
cPickle.dump(int_to_vocab, rff)
rff.close()

rff = open('./cleaned_data/word_embeddings.pkl', 'wb')
cPickle.dump(word_embedding_matrix, rff)
rff.close()


In [21]:
rf_pkl = open('./cleaned_data/vocab_to_int.pkl', 'rb')
vocab_to_int=cPickle.load(rf_pkl)
rf_pkl.close()

rf_pkl = open('./cleaned_data/int_to_vocab.pkl', 'rb')
int_to_vocab=cPickle.load(rf_pkl)
rf_pkl.close()

rf_pkl = open('./cleaned_data/word_embeddings.pkl', 'rb')
word_embedding_matrix=cPickle.load(rf_pkl)
rf_pkl.close()